In [4]:
#!/usr/bin/python3
'''
Author: Zhao-Shun Zheng, Cheng Han Yu
Date: 2021/6/19
function: Receive datas by MQTT and push datas into MySQL
email: e14051148@gs.ncku.edu.tw, n26094304@gs.ncku.edu.tw
'''

def Insert(conn, data, table = "products"):
    command = "INSERT INTO " + table \
        + " (id, OrderNumber, name, ObjectType, Pos, defection, image, Place, time_start, time_grab, time_shipment)  " \
        + "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    with conn.cursor() as cursor:
        cursor.execute(command, (data['OrderNumber'] + "_" + data['name'], 
                                data['OrderNumber'], data['name'], data['ObjectType'], data['Pos'], 
                                data['defection'], data['image'], data['Place'], 
                                data['time_start'], data['time_grab'], data['time_shipment']))
    conn.commit()
       
def Update(conn, data, table = "products"):
    for d in data:
        if d != 'id' and data[d] != "":
            with conn.cursor() as cursor:
                update_users = "UPDATE " + table + " SET " + d + " = '" + data[d] + "' WHERE id = '" + data['id'] + "'"
                cursor.execute(update_users)
                conn.commit()
                

def InsertAndUpdate(conn, data, table = "products"): 
    data['id'] = data['OrderNumber'] +"_"+  data['name']
    command = "select id from " + table + " where id=%s" 
    with conn.cursor() as cursor:
        cursor.execute(command, (data['id']))
        # gets the number of rows affected by the command executed
        r = cursor.rowcount
        if r == 0:
            Insert(conn, data, table = table)
        else:
            Update(conn, data, table = table)
        conn.commit()

In [11]:
import pymysql
import numpy as np
# 建立Connection物件
db_settings = {
    "host": "localhost",
    "port": 3306,
    "user": "root",
    "password": "f130073973",
    "db": "IOT",
    "charset": "utf8"
}
conn = pymysql.connect(**db_settings)


In [ ]:
import paho.mqtt.client as mqtt
import re
import cv2
import numpy as np
import json

def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))

    client.subscribe("IOT/Group5")
    client.subscribe("IOT/Group2")
    client.subscribe("IOT/Group6")
    client.subscribe("IOT/Group1")
    
def on_message(client, userdata, msg):
    if msg.topic == "IOT/Group5":
        data = msg.payload.decode('utf-8')
        OrderNumber = re.split(',', data)[0]
        name = re.split(',', data)[1]    
        time_start = re.split(',', data)[2]
        ObjectType = re.split(',', data)[3]
        product_info = {
            "OrderNumber": OrderNumber,
            "name": name,
            "ObjectType": ObjectType,
            "Pos": "",
            "defection": "",
            "image": "",
            "Place": "",
            "time_start": time_start,
            "time_grab": "",
            "time_shipment": "",
        }
        InsertAndUpdate(conn, product_info)
        print("G5 -> " + OrderNumber + name)
        
    if msg.topic == "IOT/Group2":
        data = msg.payload.decode('utf-8')
        OrderNumber = re.split(',', data)[0]
        name = re.split(',', data)[1]    
        p = re.split(',', data)[2]
        Pos = ""
        if int(p) == 0:
            Pos = "第一站 \n(物件取放)"
        elif int(p) == 50:
            Pos = "第二站 \n(CNC 模具製造)"
        elif int(p) == 100:
            Pos = "第三站 \n(射蠟成型)"
        elif int(p) == 150:
            Pos = "第四站 \n(瑕疵檢測)"
        elif int(p) == 200:
            Pos = "第五站 \n(分類倉儲)"
        elif int(p) == 250:
            Pos = "倉儲內"
        
            
        product_info = {
            "OrderNumber": OrderNumber,
            "name": name,
            "Pos": Pos,
        }
        InsertAndUpdate(conn, product_info)
        print("G2 -> " + OrderNumber + name)
        
    if msg.topic == "IOT/Group6":
        jsondata = json.loads(msg.payload)
        data = jsondata['payload']
        OrderNumber = re.split(',', data)[0]
        name = re.split(',', data)[1]
        Defect = re.split(',', data)[2]
        image = np.array(jsondata['image'])
        path = 'C:/Users/jenny/Desktop/Image/' + OrderNumber + "_" + name + '.jpg'
        cv2.imwrite(path, image)
        
        product_info = {
            "OrderNumber": OrderNumber,
            "name": name,
            "defection": Defect,
            "image": path,
        }
        
        InsertAndUpdate(conn, product_info)
        print("G6 -> " + OrderNumber)
        
    if msg.topic == "IOT/Group1":
        data = msg.payload.decode('utf-8')
        OrderNumber = re.split(',', data)[0]
        name = re.split(',', data)[1]
        time_grab = re.split(',', data)[2]
        time_shipment = re.split(',', data)[3]
        Place = re.split(',', data)[4]
        product_info = {
            "OrderNumber": OrderNumber,
            "name": name,
            "time_grab": time_grab,
            "time_shipment": time_shipment,
            "Place": Place,
        }
        InsertAndUpdate(conn, product_info)
        print("G1 -> " + OrderNumber)
        


        
HOST = "192.168.0.147"
PORT = 1883    
count = 0
client = mqtt.Client()

client.on_connect = on_connect

client.on_message = on_message

client.connect(HOST, PORT, 60)

client.loop_forever()

Connected with result code 0
G5 -> 210616A001
G2 -> 210616A001
G2 -> 210616A001
G2 -> 210616A001
G5 -> 210616A002
G2 -> 210616A002
G6 -> 210616
G2 -> 210616A001
G2 -> 210616A002
G1 -> 210616
G2 -> 210616A001
G2 -> 210616A002
G5 -> 210616A003
G2 -> 210616A003
G2 -> 210616A001
G6 -> 210616
G2 -> 210616A002
G2 -> 210616A003
G1 -> 210616
G2 -> 210616A002
G2 -> 210616A003
G5 -> 210616A004
G2 -> 210616A004
G2 -> 210616A002
G6 -> 210616
G2 -> 210616A003
G2 -> 210616A004
G1 -> 210616
G2 -> 210616A003
G2 -> 210616A004
G5 -> 210616A005
G2 -> 210616A005
G2 -> 210616A003
G6 -> 210616
G2 -> 210616A004
G2 -> 210616A005
G1 -> 210616
G2 -> 210616A004
G2 -> 210616A005
G5 -> 210616A006
G2 -> 210616A006
G2 -> 210616A004
G6 -> 210616
G2 -> 210616A005
G2 -> 210616A006
G1 -> 210616
G2 -> 210616A005
G2 -> 210616A006
G5 -> 210617A001
G2 -> 210617A001
G2 -> 210616A005
G6 -> 210616
G2 -> 210616A006
G2 -> 210617A001
G1 -> 210616
G2 -> 210616A006
G2 -> 210617A001
G5 -> 210617A002
G2 -> 210617A002
G2 -> 210616A006